<a href="https://colab.research.google.com/github/yolile/paraguay_oncology_medicines_ocds_data/blob/main/2022_Datos_Licitaciones_Oncol%C3%B3gicas_MSBSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descarga
Descargamos todos los procesos de contratación desde Enero del 2015 a Julio 2022, que dentro de los productos licitados tenga uno de los listados en la Lista de Medicamentos Escenciales oncológicos del Ministerio de Salud

In [1]:
# Listado de medicamentos oncologicos del LME con su código clasificador nivel 4
# de las naciones unidas utilizado para las compras realizadas a través de la DNCP
lme = [
  {'nombre': 'Ciclofosfamida', 'codigo': '51111507'},
  {'nombre':'Ifosfamida', 'codigo': '51111509'},
  {'nombre':'Metotrexato', 'codigo': '51111610'},
  {'nombre':'Mercaptopurina', 'codigo': '51111609'},
  {'nombre':'Citarabina', 'codigo': '51111602'},
  {'nombre':'Fluorouracilo', 'codigo': '51111605'},
  {'nombre':'Sulfato de vinblastina', 'codigo': '51111812'},
  {'nombre':'Sulfato de vincristina', 'codigo': '51111813'},
  {'nombre':'Etoposide', 'codigo': '51111614'},
  {'nombre':'Paclitaxel', 'codigo': '51111904'},
  {'nombre':'Clorhidrato de doxorubicina', 'codigo': '51111711'},
  {'nombre':'Sulfato de bleomicina', 'codigo': '51111701'},
  {'nombre':'Cisplatino', 'codigo': '51111506'},
  {'nombre':'Carboplatina', 'codigo': '51111503'},
  {'nombre':'Agentes Alquilantes varios', 'codigo': '51111501'},
  {'nombre':'Asparaginasa', 'codigo': '51111699'},
  {'nombre':'Hidroxiurea', 'codigo': '51111606'},
  {'nombre':'Tamoxifeno', 'codigo': '51111809'},
  {'nombre':'Anastrozol', 'codigo': '51111801'},
  {'nombre':'Filgrastim', 'codigo': '51201802'},
  {'nombre':'Mofetil micofenolato', 'codigo': '51201503'},
  {'nombre':'Ciclosporina', 'codigo': '51201502'}
]

Utilizamos el buscador de licitaciones de la DNCP que permite filtrar por fecha y productos

In [2]:

url_buscador_licitaciones = 'https://contrataciones.gov.py/buscador/licitaciones.csv?nro_nombre_licitacion=&fecha_desde=01-01-2015&fecha_hasta=31-07-2022&tipo_fecha=PUB'
url_buscador_licitaciones += '&convocante_tipo=&convocante_nombre_codigo=&codigo_contratacion=&catalogo%5Bcodigos_catalogo_n4%5D={0}&catalogo%5Bcodigos_catalogo_n4_label%5D={1}'
url_buscador_licitaciones += '&page=&order=&convocante_codigos=&convocante_tipo_codigo=&unidad_contratacion_codigo='

In [3]:
import pandas as pd
import urllib
licitaciones = pd.DataFrame()
for medicamento in lme:
  url = url_buscador_licitaciones.format(medicamento['codigo'], urllib.parse.quote(medicamento['nombre']))
  licitaciones_tmp = pd.read_csv(url, sep=';')
  licitaciones = pd.concat([licitaciones, licitaciones_tmp])

Borramos cualquier licitacion duplicada que se pudo encontrar (por ejemplo donde la misma licitacion compre más de un producto oncologico)

In [4]:
licitaciones.drop_duplicates(inplace=True)

In [5]:
licitaciones['fecha_publicacion_convocatoria'] =  pd.to_datetime(licitaciones['fecha_publicacion_convocatoria'])
licitaciones['anio'] = licitaciones['fecha_publicacion_convocatoria'].dt.year

Descargamos los detalles completos de todos los procesos de licitación en formato OCDS

In [6]:
!mkdir data

In [7]:
import requests
import json
import time

def get_ocds(ocid):
  status = 429
  if ocid is not None:
    while status == 429: 
      ocid = int(ocid)
      response = requests.get(f'http://www.contrataciones.gov.py/datos/api/v3/doc/ocds/record/ocds-03ad3f-{ocid}')
      data = response.json()
      status = response.status_code
      print(ocid, status)
      if status == 429:
        time.sleep(5)
        continue
      if status == 404:
        continue
      data = data['records'][0]['compiledRelease']
      with open(f'data/{ocid}.json', 'w') as f:
        json.dump(data, f)

licitaciones['nro_licitacion'].apply(get_ocds)

408189 200
402533 200
405414 200
406368 200
404465 200
392633 200
391512 200
396742 200
391507 200
385710 200
378092 200
373448 200
372026 200
370609 200
369451 200
360810 200
356415 200
356649 200
356306 200
352118 200
353298 200
346797 200
339483 200
339325 200
337405 200
336494 200
335127 200
323106 200
321952 200
321953 200
317872 200
317399 404
303418 200
302292 200
302685 200
301519 200
299058 200
287934 200
292542 200
285221 200
404415 200
413925 200
367573 200
327835 200
294273 200
407583 200
406068 200
408577 200
405297 200
399181 200
400628 200
397502 200
392767 200
388641 200
392024 200
389701 200
384202 200
382998 200
375756 200
366462 200
357197 200
357062 200
341420 200
343918 200
341488 200
324836 200
331593 200
326189 200
327791 200
321475 200
319372 200
315108 200
316916 200
307037 200
310862 200
303728 404
287509 200
291770 200
288269 200
405157 200
405156 200
373874 200
336861 200
301456 200
314573 200
384760 200
313130 200
408236 200
405588 200
414011 200
410210 200

0     None
1     None
2     None
3     None
4     None
      ... 
28    None
30    None
3     None
4     None
6     None
Name: nro_licitacion, Length: 127, dtype: object

In [8]:
# Instalamos unas herramientas para convertir los archivos JSON a CSV
!pip install flattentool
!pip install ocdskit
# Unimos todos los JSON en uno solo
!cat data/*.json | ocdskit package-releases > data.json
# Lo convertimos en archivos CSVs
!flatten-tool flatten data.json --root-id=ocid -f csv -s https://standard.open-contracting.org/schema/1__1__5/release-schema.json --main-sheet-name releases --root-list-path=releases   --remove-empty-schema-columns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.8 MB/s 
     |████████████████████████████████| 717 kB 7.5 MB/s 
     |████████████████████████████████| 417 kB 45.1 MB/s 
     |████████████████████████████████| 126 kB 54.6 MB/s 
     |████████████████████████████████| 251 kB 51.1 MB/s 
     |████████████████████████████████| 136 kB 58.9 MB/s 
     |████████████████████████████████| 3.6 MB 57.8 MB/s 
     |████████████████████████████████| 253 kB 59.2 MB/s 
     |████████████████████████████████| 339 kB 71.4 MB/s 
     |████████████████████████████████| 47 kB 5.0 MB/s 
  Created wheel for backports-datetime-fromisoformat: filename=backports_datetime_fromisoformat-1.0.0-cp37-cp37m-linux_x86_64.whl size=34233 sha256=349ecf06f7c2cca8daee6bdd23e64795f6bc65827f16070534a1b9baaf352963
  Stored in directory: /root/.cache/pip/wheels/b8/e4/4a/14e3dc574ae5ca155b442a3f899078f51a08b00d80ea821ff6
  Crea

Skipping field "parties/0/details/", because it has no properties.
Skipping field "tender/amendments/0/changes/0/former_value/", because it has no properties.
Skipping field "tender/amendment/changes/0/former_value/", because it has no properties.
Skipping field "awards/0/amendments/0/changes/0/former_value/", because it has no properties.
Skipping field "awards/0/amendment/changes/0/former_value/", because it has no properties.
Skipping field "contracts/0/amendments/0/changes/0/former_value/", because it has no properties.
Skipping field "contracts/0/amendment/changes/0/former_value/", because it has no properties.


# Estadísticas preliminares

## Cuántos procesos
**Se encontraron un total de 127 procesos licitatorios que compran medicamentos oncologicos de la lista de LME desde el 2015**

In [9]:
len(licitaciones)

127

## Quiénes compran?
Por número de procesos, mayormente el Ministerio de Salud, IPS, la UNA, la Policía Nacional y la DIBEN.

In [10]:
licitaciones.groupby(['convocante']).count()['nro_licitacion'].reset_index().sort_values(['nro_licitacion'], ascending=False)

,convocante,nro_licitacion
11,Ministerio de Salud Pública y Bienestar Social...,39
9,Instituto de Previsión Social / Entidades Públ...,27
4,Facultad de Ciencias Medicas / Universidad Nac...,20
12,Policia Nacional / Ministerio del Interior / P...,10
13,Secretaria Nacional por los Derechos Humanos d...,9
1,Dirección de Beneficencia / Entes Autónomos y...,7
3,Dirección de Beneficencia / Entes Autónomos y ...,3
0,Comando en Jefe Uoc 1 / Ministerio de Defensa ...,2
5,Facultad de Ciencias Quimicas / Universidad Na...,2
7,Gobierno Departamental de Concepción / Gobiern...,2


## Qué tipo de procesos se utilizan?
**Mayormente Licitaciones Públicas Nacionales y Contrataciones por Excepción**

In [11]:
licitaciones.groupby(['tipo_procedimiento']).count()['nro_licitacion'].reset_index().sort_values(['nro_licitacion'], ascending=False)

,tipo_procedimiento,nro_licitacion
3,LPN - Licitación Pública Nacional,81
1,CE - Contratación por Excepción,31
2,CO - Concurso de Ofertas,8
0,CD - Contratación Directa,7


## Análisis competencia

Preparamos los datos:
Juntamos la lista de items adjudicados con la de proveedores y clasificaciones

In [84]:
proveedores_adjudicados = pd.read_csv('flattened/awa_suppliers.csv')
adjudicaciones = pd.read_csv('flattened/awards.csv')
items = pd.read_csv('flattened/awa_items.csv')
classificacion_n4 = pd.read_csv('flattened/awa_ite_additionalClassificatio.csv')
tenders = pd.read_csv('flattened/releases.csv')
atributos = pd.read_csv('flattened/awa_ite_attributes.csv')
presentacion = atributos[(atributos['awards/0/items/0/attributes/0/name'] == 'Presentacion')]
procedencia = atributos[(atributos['awards/0/items/0/attributes/0/name'] == 'Procedencia')]
presentacion = presentacion.rename(columns={'awards/0/items/0/attributes/0/value': 'Presentacion'})
presentacion = presentacion[['awards/0/items/0/id', 'Presentacion']]
procedencia = procedencia.rename(columns={'awards/0/items/0/attributes/0/value': 'Procedencia'})
procedencia = procedencia[['awards/0/items/0/id', 'Procedencia']]

In [85]:
adjudicados = pd.merge(tenders, adjudicaciones, on='ocid')
adjudicados = pd.merge(adjudicados, proveedores_adjudicados, on='awards/0/id')
adjudicados = pd.merge(adjudicados, items, on='awards/0/id')
adjudicados = pd.merge(adjudicados, classificacion_n4, on='awards/0/items/0/id')
adjudicados = pd.merge(adjudicados, presentacion, on='awards/0/items/0/id', how='left')
adjudicados = pd.merge(adjudicados, procedencia, on='awards/0/items/0/id', how='left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ocid_x'} in the result is deprecated and will raise a MergeError in a future version.
  after removing the cwd from sys.path.


Filtramos solo los items oncológicos

In [86]:
lme_dataframe = pd.DataFrame(lme)
lme_dataframe['codigo'] = lme_dataframe['codigo'].astype('int')
items_adjudicados_oncologicos = adjudicados[adjudicados['awards/0/items/0/additionalClassifications/0/id'].isin(lme_dataframe['codigo'].tolist())]
items_adjudicados_oncologicos = items_adjudicados_oncologicos.drop_duplicates()
items_adjudicados_oncologicos['precio_x_cantidad'] = items_adjudicados_oncologicos['awards/0/items/0/unit/value/amount']*items_adjudicados_oncologicos['awards/0/items/0/quantity']

for column in ['awards/0/date', 'tender/datePublished']:
  items_adjudicados_oncologicos[column] =  pd.to_datetime(items_adjudicados_oncologicos[column])
items_adjudicados_oncologicos['anio'] = items_adjudicados_oncologicos['awards/0/date'].dt.year
items_adjudicados_oncologicos['periodo_licitacion_adjudicacion'] = (items_adjudicados_oncologicos['awards/0/date'] - items_adjudicados_oncologicos['tender/datePublished']).dt.days

## Cuántas adjudicaciones?
337 adjudicaciones, para 38 proveedores diferentes

In [87]:
len(items_adjudicados_oncologicos['awards/0/id_x'].unique())

337

In [47]:
len(items_adjudicados_oncologicos['awards/0/suppliers/0/id'].unique())

38

In [48]:
items_adjudicados_oncologicos

,ocid_x,id_x,tender/id,tender/coveredBy,tender/title,tender/status,tender/awardCriteria,tender/awardCriteriaDetails,tender/submissionMethod,tender/techniques/hasElectronicAuction,...,ocid_y,id,awards/0/id_y,awards/0/items/0/additionalClassifications/0/id,awards/0/items/0/additionalClassifications/0/scheme,awards/0/items/0/additionalClassifications/0/description,awards/0/items/0/additionalClassifications/0/uri,Presentacion,Procedencia,precio_x_cantidad
1994,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-prosaludfarma-s-a-67,51111609,UNSPSC,Mercaptopurina,https://www.contrataciones.gov.py/datos/api/v3...,UNIDAD,ARGENTINA,252000000
1995,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-prosaludfarma-s-a-67,51111606,UNSPSC,Hidroxiurea,https://www.contrataciones.gov.py/datos/api/v3...,UNIDAD,ARGENTINA,226800000
2062,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-bioethic-pharma-s-a-102,51201502,UNSPSC,Ciclosporina,https://www.contrataciones.gov.py/datos/api/v3...,UNIDAD,BRASIL,1251150000
2063,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-bioethic-pharma-s-a-102,51111701,UNSPSC,Sulfato de bleomicina,https://www.contrataciones.gov.py/datos/api/v3...,AMPOLLA,ARGENTINA,105000000
2070,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-lpn-17-15-adquisicion-medicamentos-vari...,NaN,LPN 17-15 ADQUISICION DE MEDICAMENTOS VARIOS P...,complete,priceOnly,Por Item,electronicAuction,True,...,ocds-03ad3f-288269-1,288269-lpn-17-15-adquisicion-medicamentos-vari...,288269-fusa-s-a-76,51111812,UNSPSC,Sulfato de vinblastina,https://www.contrataciones.gov.py/datos/api/v3...,AMPOLLA,URUGUAY,87000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20262,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-mather-company-srl-3,51111904,UNSPSC,Paclitaxel,https://www.contrataciones.gov.py/datos/api/v3...,NaN,ARGENTINA,2018362424
20271,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-bioethic-pharma-s-a-5,51111904,UNSPSC,Paclitaxel,https://www.contrataciones.gov.py/datos/api/v3...,NaN,ARGENTINA,2018362424
20280,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-adquisicion-medicamentos-oncologicos-ca...,NaN,ADQUISICIÓN DE MEDICAMENTOS ONCOLÓGICOS CABAZI...,complete,priceOnly,Por Total,electronicAuction,True,...,ocds-03ad3f-414011-1,414011-adquisicion-medicamentos-oncologicos-ca...,414011-tassos-sa-1-4,51111904,UNSPSC,Paclitaxel,https://w

Descargamos el CSV final con toda la información necesaria

In [89]:
items_adjudicados_oncologicos[['ocid_x', 'tender/id', 'tender/title',
                               'tender/value/amount', 'tender/datePublished',
                               'tender/procurementMethodDetails', 'tender/procuringEntity/name',
                               'buyer/name','awards/0/suppliers/0/name',
                               'awards/0/items/0/description',
       'awards/0/items/0/classification/description',
       'awards/0/items/0/quantity',
       'awards/0/items/0/unit/name',
       'awards/0/items/0/unit/value/amount',
       'Presentacion',
       'Procedencia', 'precio_x_cantidad', 'anio',
       'periodo_licitacion_adjudicacion']].to_csv('medicamentos_oncologicos.csv')

In [49]:
items_adjudicados_oncologicos.to_csv('final_completo.csv')